In [1]:
import numpy as np
import tensorflow as tf
from sklearn import datasets, model_selection
import math

In [2]:
### define hyperparameters
n_classes = 3
n_features = 4
n_hidden_1 = 10
n_hidden_2 = 20
n_hidden_3 = 10
n_hidden_4 = 20

learning_rate = 0.01
max_iter = 20000

In [3]:
### define graph
x = tf.placeholder(tf.float32, shape=[None, n_features], name="X")
y_label = tf.placeholder(tf.int32, shape=[None], name="Y_label")

# one-hot encoding
y = tf.one_hot(indices=y_label, depth=n_classes)

In [4]:
# hidden 1
with tf.name_scope('hidden1'):
  weights1 = tf.Variable(
      tf.truncated_normal([n_features, n_hidden_1],
                          stddev=1.0),
      name='weights')
  biases1 = tf.Variable(
      tf.zeros([n_hidden_1]),
      name='biases')
  hidden1 = tf.nn.relu(tf.matmul(x, weights1) + biases1)

# hidden 2
with tf.name_scope('hidden2'):
  weights2 = tf.Variable(
      tf.truncated_normal([n_hidden_1, n_hidden_2],
                          stddev=1.0),
      name='weights')
  biases2 = tf.Variable(
      tf.zeros([n_hidden_2]),
      name='biases')
  hidden2 = tf.nn.relu(tf.matmul(hidden1, weights2) + biases2)

# hidden 3
with tf.name_scope('hidden3'):
  weights3 = tf.Variable(
      tf.truncated_normal([n_hidden_2, n_hidden_3],
                          stddev=1.0),
      name='weights')
  biases3 = tf.Variable(
      tf.zeros([n_hidden_3]),
      name='biases')
  hidden3 = tf.nn.relu(tf.matmul(hidden2, weights3) + biases3)

# hidden 4
with tf.name_scope('hidden4'):
  weights4 = tf.Variable(
      tf.truncated_normal([n_hidden_3, n_hidden_4],
                          stddev=1.0),
      name='weights')
  biases4 = tf.Variable(
      tf.zeros([n_hidden_4]),
      name='biases')
  hidden4 = tf.nn.relu(tf.matmul(hidden3, weights4) + biases4)

# Linear
with tf.name_scope('softmax_linear'):
  weights5 = tf.Variable(
      tf.truncated_normal([n_hidden_4, n_classes],
                          stddev=1.0),
      name='weights')
  biases5 = tf.Variable(
      tf.zeros([n_classes]),
      name='biases')
  logits = tf.matmul(hidden4, weights5) + biases5

In [5]:
pred = tf.cast(tf.argmax(logits, 1), tf.int32)
accuracy = tf.reduce_mean( tf.cast( tf.equal(pred, y_label), tf.float32 ))

### define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [6]:
### define input data
# Load the iris dataset
iris = datasets.load_iris()

# iris has two attributes: data, target
print(iris.data.shape)
print(iris.target.shape)

# Split the data into training/testing sets
x_train, x_test, y_train, y_test = model_selection.train_test_split(
  iris.data, iris.target, test_size=0.3)

(150, 4)
(150,)


In [7]:
### Starting sessions
with tf.Session() as sess:
  ## initialize variables
  init = tf.global_variables_initializer()
  sess.run(init)

  ## training
  for i in range(max_iter):
    _, accuracy_val, pred_val, loss_val = sess.run(
      [train_step, accuracy, pred, loss], 
      feed_dict={x: x_train, y_label: y_train})

    if i % 1000 == 0:
      print('=========== Epoch: %d ===========' % i)
      print('Loss', loss_val)
      print('Accuracy', accuracy_val)
      print('Y_prediction ', pred_val[:10])
      print('True', y_train[:10])
      
      # accuracy for testset
      test_accuracy, test_pred = sess.run( 
        [accuracy, pred],
        feed_dict={x: x_test, y_label: y_test})
      print('---- evaluation ----')
      print('acc: %.4f' % test_accuracy)
      print('pred', test_pred)
      print('true', y_test)


    if math.isnan(loss_val):
      print('LOSS is NAN!')
      break


=========== Epoch: 0 ===========
Loss 40.8149
Accuracy 0.228571
Y_prediction  [0 0 0 0 0 0 0 0 0 0]
True [2 2 2 1 0 1 2 2 2 2]
---- evaluation ----
acc: 0.2000
pred [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2]
true [2 1 2 0 1 0 0 2 0 0 2 0 1 2 2 2 0 0 1 0 1 0 2 0 0 1 1 1 0 1 0 1 0 0 2 0 1
 1 1 0 0 1 1 1 0]
=========== Epoch: 1000 ===========
Loss 1.09027
Accuracy 0.390476
Y_prediction  [2 2 2 2 2 2 2 2 2 2]
True [2 2 2 1 0 1 2 2 2 2]
---- evaluation ----
acc: 0.2000
pred [2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 1]
true [2 1 2 0 1 0 0 2 0 0 2 0 1 2 2 2 0 0 1 0 1 0 2 0 0 1 1 1 0 1 0 1 0 0 2 0 1
 1 1 0 0 1 1 1 0]
=========== Epoch: 2000 ===========
Loss 1.09026
Accuracy 0.390476
Y_prediction  [2 2 2 2 2 2 2 2 2 2]
True [2 2 2 1 0 1 2 2 2 2]
---- evaluation ----
acc: 0.2000
pred [2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 1]
true [2 1 2 0 1 0 0 2 0 